## Imports

In [1]:
import pandas as pd
import json

In [63]:
def get_bomb_plant_index(frames: list) -> int:
    for index, frame in enumerate(frames):
        if frame["bombPlanted"] == True:
            return index

def was_round_t_success(round, fop):
    vs_eco = round["ctBuyType"] == "Full Eco"
    half_vs_half = round["tBuyType"] == "Semi Eco" and round["ctBuyType"] == "Semi Eco"
    force_and_full_vs_eco_or_half = round["tBuyType"] not in ['Full Eco', 'Semi Eco'] and round["ctBuyType"] in ['Full Eco', 'Semi Eco']
    if round["roundEndReason"] in ['TerroristsWin', 'TargetBombed']:
        return True
    if round["roundEndReason"] == 'TargetSaved':
        return False
    if round["bombPlantTick"] != None:
        if vs_eco or half_vs_half or force_and_full_vs_eco_or_half:
            return False
        if fop['ct']["alivePlayers"] - fop['t']["alivePlayers"] <= 1:
            return True
    return False


def bomb_last_bombsite(round):
    A = ['BombsiteA', 'Graveyard', 'Pit', 'TopofMid', 'Arch']
    B = ['BombsiteB', 'Banana', 'CTSpawn', 'Quad', 'Ruins']
    lastBombBomsite = None
    for frame in round['frames']:
        for object in frame['world']:
            if object['objectType'] == 'bomb':
                if object['areaName'] in A:
                    lastBombBomsite = 'A'
                elif object['areaName'] in B:
                    lastBombBomsite = 'B'
    return lastBombBomsite


def frame_index_of_last_death(round):
    def find_player(players, player_name):
        for player in players:
            if player['name'] == player_name:
                return player
        return None

    last_index = None
    A = ['BombsiteA', 'Graveyard', 'Pit', 'TopofMid', 'Arch']
    B = ['BombsiteB', 'Banana', 'CTSpawn', 'Quad', 'Ruins']
    for (index, frame) in enumerate(round['frames']):
        for player in frame['t']['players']:
            if player['hp'] == 0 and find_player(round['frames'][index - 1]['t']['players'], player['name'])['hp'] > 0:
                if player['areaName'] in A or player['areaName'] in B:
                    last_index = index
    return last_index

In [71]:
parsed_demo_path = './parsed_demos/astralis-vs-nip-m2-inferno (2).json'
data = json.load(open(parsed_demo_path))
rounds = []

for round in data['gameRounds']:
    if (not round["isWarmup"]
    and round["frames"]
    and len(round["frames"]) > 30
    and round["weaponFires"]
    and round["winningTeam"]):
        if round["bombPlantTick"]:
            bomb_plant_index = get_bomb_plant_index(round["frames"])
        else:
            bomb_plant_index = len(round["frames"]) - 1
        fop = round["frames"][bomb_plant_index]
        foi = round["frames"][bomb_plant_index - 30]

        if (foi["t"]["players"]
        and len(foi["t"]["players"]) == 5
        and foi["ct"]["players"]
        and len(foi["ct"]["players"]) == 5):
            round = {
                'round_number': round["endTScore"] + round["endCTScore"],
                'bombsite': fop["bombsite"] or bomb_last_bombsite(round),
                'was_round_t_success': was_round_t_success(round, fop),
                'round_end_reason': round["roundEndReason"],
                'index_of_last_death': frame_index_of_last_death(round),
            }
            rounds.append(round)

df = pd.DataFrame(rounds)
df.to_csv("bomb_test.csv", index=False)
    

In [72]:
data_df = pd.read_csv("bomb_test.csv")
data_df

,round_number,bombsite,was_round_t_success,round_end_reason,index_of_last_death
0,1,A,True,TerroristsWin,63.0
1,2,A,True,TerroristsWin,118.0
2,3,B,True,TargetBombed,98.0
3,4,A,False,CTWin,104.0
4,5,A,True,BombDefused,106.0
5,6,A,False,CTWin,63.0
6,7,B,True,TerroristsWin,110.0
7,8,B,True,TargetBombed,NaN
8,9,A,True,TargetBombed,75.0
9,10,A,False,CTWin,110.0
